# Create the Database
This is a small _RDBMS_ created to hold the text based data of the artworks.

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
data_file_path = "./data/"

In [3]:
combined_artists = pd.read_csv("".join([data_file_path,"combined_artists/combined_artists.csv"]))

## ER Diagram <br/>
The Artists metadata database. <br/>
<img src="./images/artists_er_diagram.png" width="1280px"><br/>
__NOTE:__ as we are dealing with a small dataset and as this is not a _database design_ course. I shall keep the design as simple as possible. If the entire catalog of the _wikiart.org_ website were to be used. Then performance modifications could be made to both the database design and the query statements throughout the rest of the project. 

## The creation of DataFrames holding the data to be placed into the RDBMS Tables
First we will construct DataFrames containing the key fields using the flexibility of the DataFrame to allow data manipulation where needed. Once the data is in a form ready for the RDBMS we can construct the tables and populate them. 

## Create the _artist_table_data_ Dataframe
The primary list of Artists.

In [4]:
# create the dataframe
artist_table_data             = pd.DataFrame(sorted(combined_artists["artist"].unique()), columns = ["name"])
artist_table_data.index.names = ["id"]

## Create the _genre_table_data_ Dataframe
The primary list of genres.

In [5]:
# create the dataframe
genre_table_data             = pd.DataFrame(sorted(combined_artists["genre"].unique()), columns = ["genre"])
genre_table_data.index.names = ["id"]

## Create the _style_table_data_ Dataframe
The primary list of Genres.

In [6]:
# create the dataframe
style_table_data             = pd.DataFrame(sorted(set(str(combined_artists["style"].unique()).replace("\"[", "").replace("]\"", "").replace("\n", "").replace("' '", ",").replace("', '", ",").replace("['", "").replace("']", "").split(","))), columns = ["style"])
style_table_data.index.names = ["id"]

## Create the _artwork_table_data_ Dataframe
Key data about the Artwork:
- The artist
- The genre
- The artworks title
- year of completion

In [7]:
# create the dataframe
artwork_table_data                      = pd.DataFrame()
artwork_table_data[        "artist_id"] = combined_artists[           "artist"].apply(lambda x: artist_table_data["name"][artist_table_data["name"] == x].index[0])
artwork_table_data["artist_artwork_id"] = combined_artists["artist_artwork_id"]
artwork_table_data[         "genre_id"] = combined_artists[            "genre"].apply(lambda x: genre_table_data["genre"][genre_table_data["genre"] == x].index[0])
artwork_table_data[            "title"] = combined_artists[            "title"]
artwork_table_data[             "year"] = combined_artists[             "year"]
artwork_table_data.index.names          = [                               "id"]

## Create the _artwork_style_table_data_ Dataframe
Linking a artwork to the list of styles. An artwork can be considered to have more than one style. This table provides that mapping to allow this _Many-to-Many_ deffinition.

In [8]:
# create the dataframe
artwork_style_table_data = pd.DataFrame()

# loop through all values withing the combined_artists dataframes index
for artwork_id in combined_artists.index:
    
    # select the data row for the current index value
    row_data = combined_artists.iloc[artwork_id]
    
    # split the style column and loop throught each value (normally only one). when we
    # find multiple values we generate a row for each value
    for artworks_style in str(row_data["style"]).replace("[", "").replace("]", "").replace("''", "").replace("'", "").split(","):
        
        # lookup the style_id in the previously created style_table_data dataframe
        style_id = style_table_data[style_table_data["style"] == artworks_style.strip()].index[0]
        
        # create a dict containing just the artwork_id and the style_id
        row_data = {"artwork_id" : artwork_id,
                    "style_id"   : style_id}
        
        # append the dict as a row to the artwork_style_table_data dataframe
        artwork_style_table_data = artwork_style_table_data.append(row_data, ignore_index = True)

# force both data columns of the dataframe to be integers
artwork_style_table_data = artwork_style_table_data.astype('int32')

## Create the _artwork_image_table_data_ Dataframe
Contains the __URL__ of the image on the _WikiArt_ webserver. The __filename__ of its file held locally on the PC filesystem and the __IMAGE_TAG__ used for identifying the image in the code.

In [9]:
# create the dataframe
artwork_image_table_data = pd.DataFrame()

# get the unique artwork_id from the combined_artists index
artwork_image_table_data[           "artwork_id"] = combined_artists.index

# create a temporary column to hold the artist_id
artwork_image_table_data[        "tmp_artist_id"] = combined_artists["artist"].apply(lambda x: artist_table_data["name"][artist_table_data["name"] == x].index[0])

# create a temporary column to hold the artist_artwork_id
artwork_image_table_data["tmp_artist_artwork_id"] = combined_artists["artist_artwork_id"]
artwork_image_table_data["tmp_artist_artwork_id"] = artwork_image_table_data["tmp_artist_artwork_id"].astype('int32')

# generate the image filename
artwork_image_table_data[       "image_filename"] = artwork_image_table_data.apply(lambda x: "full_image_{:02}_{:04}.jpg".format(x["tmp_artist_id"],x["tmp_artist_artwork_id"] ), axis=1)
  
# generate the image tag
artwork_image_table_data[            "image_tag"] = artwork_image_table_data.apply(lambda x: "{:02}_{:04}".format(               x["tmp_artist_id"],x["tmp_artist_artwork_id"] ), axis=1)
    
# add the url of the image
artwork_image_table_data[                  "url"] = combined_artists["url"]    
    
# delete the temporary columns
artwork_image_table_data.drop(["tmp_artist_id", "tmp_artist_artwork_id"], axis = 1, inplace = True)

## Create the _artist_style_table_data_ Dataframe
This table contains the _Style Profile_ of each artist. It holds details of each style that the artist has been known to have used and a percentage of their body of work that is in this style.

In [10]:
# create the dataframe
artist_style_table_data = pd.DataFrame()

# loop through all values withing the combined_artists dataframes index
for artwork_id in combined_artists.index:

    try:
        # select the data row for the current index value
        row_data = combined_artists.iloc[artwork_id]

        # split the style column and loop throught each value (normally only one). when we
        # find multiple values we generate a row for each value
        for artwork_style in str(row_data["style"]).replace("[", "").replace("]", "").replace("''", "").replace("'", "").split(","):

            # create a dict containing just the artwork_id and the style_id
            row_data = {"artist" : row_data["artist"],
                        "style"   : artwork_style}

            # append the dict as a row to the artwork_style_table_data dataframe
            artist_style_table_data = artist_style_table_data.append(row_data, ignore_index = True)
    except:
        pass

# count the number of artworks per artist
artist_total_style = pd.DataFrame(artist_style_table_data.groupby(["artist"]).count()["style"]).rename(columns={"style" : "count_total"})
artist_total_style.reset_index(inplace = True)

# set up the count column
artist_style_table_data["count"] = 0

# count the number of artworks by style for each artist
artist_style_table_data = pd.DataFrame(artist_style_table_data.groupby(["artist", "style"]).count()["count"]).rename(columns={"count_total" : "count"})
artist_style_table_data.reset_index(inplace = True)

# add the total artworks as a coulmn to the number of 
# artworks by genre for each artist dataset
artist_style_table_data = artist_style_table_data.merge(artist_total_style)

# calculate the percentage of the artists artworks that
# are of the genre
artist_style_table_data["percentage"] = ((100 / artist_style_table_data["count_total"]) * artist_style_table_data["count"]).round(2)

# replace text values with id values
artist_style_table_data[    "artist"] = artist_style_table_data["artist"].apply(lambda x: artist_table_data["name"][artist_table_data["name"] == x.strip()].index[0])
artist_style_table_data[     "style"] = artist_style_table_data["style"].apply(lambda  x: style_table_data["style"][style_table_data["style"] == x.strip()].index[0])

# modify the column names
artist_style_table_data.rename(columns={"artist" : "artist_id", "style" : "style_id"}, inplace = True)

# drop the total_count column
artist_style_table_data.drop(columns={"count_total"}, inplace = True)

## Create the _artist_genre_table_data_ Dataframe
This table contains the _Genre Profile_ of each artist. It holds details of each genre that the artist has been known to have used and a percentage of their body of work that is of this genre.

In [11]:
# create the dataframe
artist_genre_table_data = pd.DataFrame()

# count the number of artworks per artist
artist_total_artworks = pd.DataFrame(combined_artists.groupby(["artist"]).count()["title"]).rename(columns={"title" : "count_total"})
artist_total_artworks.reset_index(inplace = True)

# count the number of artworks by genre for each artist
artist_genre_table_data = pd.DataFrame(combined_artists.groupby(["artist", "genre"]).count()["title"]).rename(columns={"title" : "count"})
artist_genre_table_data.reset_index(inplace = True)

# add the total artworks as a coulmn to the number of 
# artworks by genre for each artist dataset
artist_genre_table_data = artist_genre_table_data.merge(artist_total_artworks)

# calculate the percentage of the artists artworks that
# are of the genre
artist_genre_table_data["percentage"] = ((100 / artist_genre_table_data["count_total"]) * artist_genre_table_data["count"]).round(2)

# replace text values with id values
artist_genre_table_data[    "artist"] = artist_genre_table_data["artist"].apply(lambda x: artist_table_data["name" ][artist_table_data["name"] == x.strip()].index[0])
artist_genre_table_data[     "genre"] = artist_genre_table_data["genre" ].apply(lambda x: genre_table_data[ "genre"][genre_table_data["genre"] == x.strip()].index[0])

# modify the column names
artist_genre_table_data.rename(columns={"artist" : "artist_id", "genre" : "genre_id"}, inplace = True)

# drop the total_count column
artist_genre_table_data.drop(columns={"count_total"}, inplace = True)

## Build the Database file
It will be automatically created if it does not already exist

In [12]:
# Create a DB connection between python and the file system
conn = sqlite3.connect(''.join([data_file_path,"/database/artist.db"]))

## Create and Popluate the Database Tables
Now we are ready to create the tables and populate them with the data.

In [13]:
# Convert the dataFrames to DB tables.
artist_table_data.to_sql(       "ARTIST"       , conn, if_exists = "replace")
genre_table_data.to_sql(        "GENRE"        , conn, if_exists = "replace")
style_table_data.to_sql(        "STYLE"        , conn, if_exists = "replace")
artwork_table_data.to_sql(      "ARTWORK"      , conn, if_exists = "replace")
artwork_style_table_data.to_sql("ARTWORK_STYLE", conn, if_exists = "replace")
artwork_image_table_data.to_sql("ARTWORK_IMAGE", conn, if_exists = "replace")
artist_style_table_data.to_sql( "ARTIST_STYLE" , conn, if_exists = "replace")
artist_genre_table_data.to_sql( "ARTIST_GENRE" , conn, if_exists = "replace")